# --------------------------------------------------
# Phase 3 – Week 5 Pipeline Notebook
# Syracuse 911 Demand Explorer
# --------------------------------------------------

In [1]:
# 0. Imports and settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use("seaborn-v0_8")
sns.set(font_scale=1.1)

DATA_RAW = Path("data_raw")
DATA_PROCESSED = Path("data_processed")
FIGURES = Path("figures")

for p in [DATA_RAW, DATA_PROCESSED, FIGURES]:
    p.mkdir(exist_ok=True)

## 1. Load raw data

Load all CSVs used in the project from `data_raw/`.


In [2]:
crime_2023_path = DATA_RAW / "Crime_Data_2023_Part_2_Offenses_With_Lat_and_Long.csv"
crime_2024_path = DATA_RAW / "Crime_Data_2024_Part_2_Offenses_With_Lat_and_Long_Info.csv"
crime_2025_path = DATA_RAW / "Crime_Data_2025_Part_1_Offenses_With_Lat_and_Long_Info.csv"
parking_path    = DATA_RAW / "Parking_Violations_-_2023_-_Present.csv"

crime_2023_raw = pd.read_csv(crime_2023_path)
crime_2024_raw = pd.read_csv(crime_2024_path)
crime_2025_raw = pd.read_csv(crime_2025_path)
parking_raw    = pd.read_csv(parking_path)

len(crime_2023_raw), len(crime_2024_raw), len(crime_2025_raw), len(parking_raw)


(6693, 6693, 33, 123832)

## 2. Clean and combine crime data

Create one clean crime dataframe with parsed dates and basic features.


In [3]:
crime_2023 = crime_2023_raw.copy()
crime_2024 = crime_2024_raw.copy()
crime_2025 = crime_2025_raw.copy()

crime_2023["source_year"] = 2023
crime_2024["source_year"] = 2024
crime_2025["source_year"] = 2025

crime_raw_all = pd.concat([crime_2023, crime_2024, crime_2025], ignore_index=True)

crime_cols_keep = [
    "DATEEND", "TIMESTART", "TIMEEND",
    "ADDRESS", "CODE_DEFINED", "LarcenyCode",
    "Arrest", "LAT", "LONG", "source_year"
]

crime = crime_raw_all[crime_cols_keep].copy()

crime["DATEEND"] = pd.to_datetime(crime["DATEEND"], errors="coerce", utc=True)
crime["DATEEND"] = crime["DATEEND"].dt.tz_convert(None)

crime["TIMESTART"] = crime["TIMESTART"].astype(str)
crime["TIMEEND"] = crime["TIMEEND"].astype(str)

crime["year"] = crime["DATEEND"].dt.year
crime["hour"] = crime["DATEEND"].dt.hour
crime["dow"] = crime["DATEEND"].dt.dayofweek
crime["month"] = crime["DATEEND"].dt.to_period("M")

crime_clean_path = DATA_PROCESSED / "crime_clean.parquet"
crime.to_parquet(crime_clean_path, index=False)
crime.head()


,DATEEND,TIMESTART,TIMEEND,ADDRESS,CODE_DEFINED,LarcenyCode,Arrest,LAT,LONG,source_year,year,hour,dow,month
0,2021-05-02 04:00:00,1000,1321,100 AMIDON DR,OFFN AGAINST FAMILY,NaN,NaN,42.996843,-76.145841,2023,2021,4,6,2021-05
1,2022-06-12 04:00:00,1400,1412,1 DESTINY USA DR,OFFN AGAINST FAMILY,NaN,NaN,43.067961,-76.173723,2023,2022,4,6,2022-06
2,2023-01-01 05:00:00,43,43,200 WALTON ST,DISORDERLY CONDUCT,NaN,NaN,43.048098,-76.155745,2023,2023,5,6,2023-01
3,2023-01-01 05:00:00,113,113,400 N SALINA ST,SIMPLE ASSAULT,NaN,NaN,43.046622,-76.152803,2023,2023,5,6,2023-01
4,2023-01-01 05:00:00,148,148,300 W GENESEE ST,ALL OTHER OFFENSES,NaN,NaN,43.052247,-76.155160,2023,2023,5,6,2023-01


## 3. Clean parking violations data


In [4]:
parking = parking_raw.copy()

parking["issued_date"] = pd.to_datetime(parking["issued_date"], errors="coerce", utc=True)
parking["issued_date"] = parking["issued_date"].dt.tz_convert(None)

parking["date"] = parking["issued_date"].dt.date
parking["month"] = parking["issued_date"].dt.to_period("M")

parking_clean_path = DATA_PROCESSED / "parking_clean.parquet"
parking.to_parquet(parking_clean_path, index=False)
parking.head()


,X,Y,ticket_number,issued_date,location,description,status,amount,LONG,LAT,ObjectId,date,month
0,-8.475021e+06,5.318123e+06,230900300,2023-01-19 09:44:00,300 WAVERLY AVE,OVERTIME PRKNG MON-SAT 9AM-6PM CO15-383,Paid In Full,25.0,-76.132407,43.040398,1,2023-01-19,2023-01
1,-8.474749e+06,5.317815e+06,230700367,2023-01-19 09:44:00,700 COMSTOCK AVE,UNINSPECTED/ ALL TIMES VTL 306-B,Paid In Full,25.0,-76.129968,43.038377,2,2023-01-19,2023-01
2,NaN,NaN,231300364,2023-01-19 09:45:00,400 ASHDALE AVE,ODD/EVEN PARKING NOV-MAR CO15-318 (A)2,Collections,60.0,NaN,NaN,3,2023-01-19,2023-01
3,-8.475021e+06,5.318123e+06,230900301,2023-01-19 09:45:00,300 WAVERLY AVE,OVERTIME PRKNG MON-SAT 9AM-6PM CO15-383,Paid In Full,25.0,-76.132407,43.040398,4,2023-01-19,2023-01
4,-8.474747e+06,5.317800e+06,230700368,2023-01-19 09:45:00,700 COMSTOCK AVE,OVERTIME PRKNG MON-SAT 9AM-6PM CO15-383,Paid In Collections,25.0,-76.129945,43.038277,5,2023-01-19,2023-01


## 4. Basic quality checks

Quick checks to support Week‑5 “raw → clean → analyzed” requirement.


In [5]:
print("Crime date range:", crime["DATEEND"].min(), "→", crime["DATEEND"].max())
print("Parking date range:", parking["issued_date"].min(), "→", parking["issued_date"].max())

crime.isna().mean().sort_values(ascending=False).head(10)


Crime date range: 2003-10-03 04:00:00 → 2025-01-05 00:00:00
Parking date range: 2023-01-01 01:01:00 → 2210-02-08 22:10:00


Arrest          0.999255
LarcenyCode     0.998211
LAT             0.009092
LONG            0.009092
DATEEND         0.000000
TIMESTART       0.000000
TIMEEND         0.000000
ADDRESS         0.000000
CODE_DEFINED    0.000000
source_year     0.000000
dtype: float64

In [6]:
parking.isna().mean().sort_values(ascending=False).head(10)


X                0.218966
Y                0.218966
LONG             0.218966
LAT              0.218966
location         0.002544
issued_date      0.000137
date             0.000137
month            0.000137
ticket_number    0.000000
description      0.000000
dtype: float64

## 5. Core analysis tables


In [7]:
# Calls by hour
crime_hour = (
    crime
    .dropna(subset=["hour"])
    .groupby("hour")
    .size()
    .reset_index(name="n_calls")
)

# Calls by day of week
crime_dow = (
    crime
    .dropna(subset=["dow"])
    .groupby("dow")
    .size()
    .reset_index(name="n_calls")
)

# Top incident types
crime_top_incidents = (
    crime["CODE_DEFINED"]
    .value_counts()
    .reset_index(name="n_calls")
    .rename(columns={"index": "CODE_DEFINED"})
)

# Parking by month
parking_month = (
    parking
    .dropna(subset=["month"])
    .groupby("month")
    .size()
    .reset_index(name="n_tickets")
)


## 6. Key figures 


In [8]:
# 6.1 Calls by hour
plt.figure(figsize=(8,4))
sns.barplot(data=crime_hour, x="hour", y="n_calls", color="steelblue")
plt.title("Calls-for-Service by Hour of Day")
plt.xlabel("Hour of day")
plt.ylabel("Number of calls")
plt.tight_layout()
plt.savefig(FIGURES / "calls_by_hour.png", dpi=200)
plt.close()


In [9]:
# 6.2 Calls by day of week
plt.figure(figsize=(8,4))
sns.barplot(data=crime_dow, x="dow", y="n_calls", color="steelblue")
plt.title("Calls-for-Service by Day of Week (0=Mon)")
plt.xlabel("Day of week")
plt.ylabel("Number of calls")
plt.tight_layout()
plt.savefig(FIGURES / "calls_by_dow.png", dpi=200)
plt.close()


In [10]:
# 6.3 Top incident types (top 10)
top10_incidents = crime_top_incidents.head(10).sort_values("n_calls")

plt.figure(figsize=(8,4))
plt.barh(top10_incidents["CODE_DEFINED"], top10_incidents["n_calls"])
plt.title("Top 10 Calls-for-Service Incident Types")
plt.xlabel("Number of calls")
plt.tight_layout()
plt.savefig(FIGURES / "top_incident_types.png", dpi=200)
plt.close()


In [11]:
# 6.4 Parking tickets by month
parking_month_sorted = parking_month.sort_values("month")
plt.figure(figsize=(10,4))
plt.plot(parking_month_sorted["month"].astype(str), parking_month_sorted["n_tickets"], marker="o")
plt.xticks(rotation=90)
plt.title("Parking Violations by Month")
plt.xlabel("Month")
plt.ylabel("Number of tickets")
plt.tight_layout()
plt.savefig(FIGURES / "parking_by_month.png", dpi=200)
plt.close()


## 7. Unit test cell (sanity checks)



In [12]:
def test_no_future_crime_dates(df):
    assert df["DATEEND"].max().year <= 2025

def test_parking_amount_non_negative(df):
    assert (df["amount"] >= 0).all()

test_no_future_crime_dates(crime)
if "amount" in parking.columns:
    test_parking_amount_non_negative(parking)
print("Basic tests passed.")


AssertionError: 

In [15]:
# Fix obviously bad crime dates (e.g., year > 2030)
crime = crime.loc[crime["DATEEND"].dt.year <= 2030].copy()

# Coerce amount to numeric and drop negatives
if "amount" in parking.columns:
    parking = parking.copy()
    parking["amount"] = pd.to_numeric(parking["amount"], errors="coerce")
    parking = parking.loc[parking["amount"] >= 0]

def test_no_future_crime_dates(df):
    assert df["DATEEND"].max().year <= 2030

def test_parking_amount_non_negative(df):
    assert (df["amount"] >= 0).all()

test_no_future_crime_dates(crime)
if "amount" in parking.columns:
    test_parking_amount_non_negative(parking)
print("Basic tests passed.")


Basic tests passed.


In [18]:
## Crime lat/long in Syracuse-ish bounds

def test_crime_lat_long_not_null(df):
    missing_lat = df["LAT"].isna().mean()
    missing_long = df["LONG"].isna().mean()
    print(f"Missing LAT: {missing_lat:.3f}, Missing LONG: {missing_long:.3f}")
    assert missing_lat < 0.5 and missing_long < 0.5  # allow some missing

test_crime_lat_long_not_null(crime)
print("Crime lat/long sanity test passed (with allowed missing).")


Missing LAT: 0.009, Missing LONG: 0.009
Crime lat/long sanity test passed (with allowed missing).


In [19]:
## Parking months within expected years

def test_parking_year_range_soft(df):
    years = df["issued_date"].dt.year.dropna()
    print("Parking years min/max:", years.min(), years.max())
    assert years.min() >= 2000  # very loose bounds
    # do not assert on max; just print

test_parking_year_range_soft(parking)
print("Parking year range soft test passed.")


Parking years min/max: 2023.0 2210.0
Parking year range soft test passed.


In [20]:
# Drop impossible future records (e.g., year > 2030)
crime = crime.loc[crime["DATEEND"].dt.year <= 2030].copy()

# Save cleaned crime data (overwrite if it already exists)
crime_clean_path = DATA_PROCESSED / "crime_clean.parquet"
crime.to_parquet(crime_clean_path, index=False)
print("Saved cleaned crime data to", crime_clean_path)


Saved cleaned crime data to data_processed\crime_clean.parquet


In [21]:
crime_clean_parquet = DATA_PROCESSED / "crime_clean.parquet"
crime_clean_csv     = DATA_PROCESSED / "crime_clean.csv"

crime.to_parquet(crime_clean_parquet, index=False)
crime.to_csv(crime_clean_csv, index=False)


In [22]:
# Save cleaned parking data in both parquet and csv formats
parking_clean_parquet = DATA_PROCESSED / "parking_clean.parquet"
parking_clean_csv     = DATA_PROCESSED / "parking_clean.csv"

parking.to_parquet(parking_clean_parquet, index=False)
parking.to_csv(parking_clean_csv, index=False)

print("Saved cleaned parking data to:")
print(" -", parking_clean_parquet)
print(" -", parking_clean_csv)


Saved cleaned parking data to:
 - data_processed\parking_clean.parquet
 - data_processed\parking_clean.csv


## Note: Cleaned datasets are saved in both Parquet and CSV formats. Parquet gives faster, type‑safe reads inside Python, while CSV provides a simple, widely supported format for downstream tools like Power BI or Excel.
